<a href="https://colab.research.google.com/github/simecek/PseudoDNA_Generator/blob/master/data/Random_Transcript_Seqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Random Transcript Sequences


## Setup

Installation for colab environment.

In [1]:
!pip install biopython pyensembl

     |████████████████████████████████| 2.3MB 2.8MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
  Created wheel for pyensembl: filename=pyensembl-1.8.7-cp36-none-any.whl size=56339 sha256=16120b8e9b8b73edd00f6d42385c305d3d8d39153b6f0953049357c2ecedd24e
  Stored in directory: /root/.cache/pip/wheels/69/8b/4d/a6c76aa39aa3619c8b8c5ff3e09cd180df57e5e9f04c655794
  Created wheel for typechecks: filename=typechecks-0.1.0-cp36-none-any.whl size=2770 sha256=554931b7d62a82ffbd4325faab02206bbd6ad5740b618a4621e80d00e1c9d0c4
  Stored in directory: /root/.cache/pip/wheels/27/ad/82/4a9bbc88f1d31bb1a0c053e29efafc02f56e2ed1598f5639b2
  Created wheel for datacache: filename=datacache-1.1.5-cp36-none-any.whl size=16472 sha256=bdaa0a465c454000f6ea3d5c864ebe774b3080a96cc6a016a2e1833406a18c0d
  Stored in directory: /root/.cache/pip/wheels/6b/89/37/76d23043c61b5096dd0170fd6ea2f6474b38d69965b633118a
  Created wheel for memoized-property: filen

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pyensembl install --release 97 --species human

2020-06-26 16:38:58,348 - pyensembl.shell - INFO - Running 'install' for EnsemblRelease(release=97, species='homo_sapiens')
2020-06-26 16:38:58,348 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz from URL ftp://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz
2020-06-26 16:38:58,349 - datacache.download - INFO - Downloading ftp://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz to /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz
2020-06-26 16:39:01,389 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL ftp://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
2020-06-26 16:39:01,389 - datacache.download - INFO - Downloading ftp://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to /root/.

In [77]:
import pandas as pd
import numpy as np
import gzip
from tqdm.notebook import tqdm

from Bio import SeqIO   # for reading fasta files
from pyensembl import EnsemblRelease   # to get the gene list

ENSEMBL_RELEASE = 97
DNA_TOPLEVEL_FASTA_PATH = "/content/drive/My Drive/data/ensembl/Homo_sapiens.GRCh38.dna.toplevel.fa.gz"

# to generate random sequences
N = 50_000    # how many
K = 200       # how long
OUTPUT_FILE = '/content/drive/My Drive/data/random/random_transcripts.csv'   # where to save them

CHRS = [str(chr) for chr in range(1,23)] + ['X', 'Y', 'MT']

## Get transcript list

In [34]:
# release 97 uses human reference genome GRCh38
data = EnsemblRelease(ENSEMBL_RELEASE)

In [36]:
human_transcripts = data.transcript_ids()
len(human_transcripts)

226788

In [37]:
human_transcripts[0], data.transcript_by_id(human_transcripts[0])


('ENST00000000233',
 Transcript(transcript_id='ENST00000000233', transcript_name='ARF5-201', gene_id='ENSG00000004059', biotype='protein_coding', contig='7', start=127588411, end=127591700, strand='+', genome='GRCh38'))

In [38]:
transcripts_full_info  = [data.transcript_by_id(transcript) for transcript in human_transcripts]

In [54]:
human_transcript_tuples = [(x.transcript_id, x.gene_id, x.biotype, x.contig, x.start, x.end, x.strand, x.coding_sequence) for x in transcripts_full_info if (x.biotype == 'protein_coding') & x.contains_start_codon & x.contains_stop_codon]
human_transcript_table = pd.DataFrame.from_records(human_transcript_tuples, columns=["id", "gene_id", "biotype", "chr", "start", "end", "strand", "coding_sequence"])
assert all(human_transcript_table.start <= human_transcript_table.end)

human_transcript_table.head()

,id,gene_id,biotype,chr,start,end,strand,coding_sequence
0,ENST00000000233,ENSG00000004059,protein_coding,7,127588411,127591700,+,ATGGGCCTCACCGTGTCCGCGCTCTTTTCGCGGATCTTCGGGAAGA...
1,ENST00000000412,ENSG00000003056,protein_coding,12,8940361,8949645,-,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...
2,ENST00000000442,ENSG00000173153,protein_coding,11,64305524,64316743,+,ATGTCCAGCCAGGTGGTGGGCATTGAGCCTCTCTACATCAAGGCAG...
3,ENST00000001008,ENSG00000004478,protein_coding,12,2794970,2805423,+,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...
4,ENST00000001146,ENSG00000003137,protein_coding,2,72129238,72148038,-,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...


In [68]:
assert ~human_transcript_table.coding_sequence.str.contains('N').any()

human_transcript_table['length'] = human_transcript_table.coding_sequence.apply(len)
selected_regions = human_transcript_table[human_transcript_table.length > K].copy()

human_transcript_table.shape, selected_regions.shape

((57208, 9), (55953, 9))

In [73]:
selected_regions['random_start'] = [np.random.randint(c_len-200) for c_len in selected_regions.length]
selected_regions['random_end'] = selected_regions['random_start'] + K - 1
selected_regions['seq'] = ''
for i in range(selected_regions.shape[0]):
  selected_regions['seq'].iloc[i] = (selected_regions['coding_sequence'].iloc[i])[selected_regions['random_start'].iloc[i]:selected_regions['random_start'].iloc[i]+200]
selected_regions.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,id,gene_id,biotype,chr,start,end,strand,coding_sequence,length,random_start,seq,random_end
0,ENST00000000233,ENSG00000004059,protein_coding,7,127588411,127591700,+,ATGGGCCTCACCGTGTCCGCGCTCTTTTCGCGGATCTTCGGGAAGA...,543,208,GCCAGGACAAGATTCGGCCTCTGTGGCGGCACTACTTCCAGAACAC...,407
1,ENST00000000412,ENSG00000003056,protein_coding,12,8940361,8949645,-,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...,834,203,AGACACATACATCTACATCTTCAGGGTGTGCCGGGAAGCTGGCAAC...,402
2,ENST00000000442,ENSG00000173153,protein_coding,11,64305524,64316743,+,ATGTCCAGCCAGGTGGTGGGCATTGAGCCTCTCTACATCAAGGCAG...,1272,821,CCAGCGCTCACTGCCACTGCAGGATGAGCTGGCCTTCGCTGAGGAC...,1020
3,ENST00000001008,ENSG00000004478,protein_coding,12,2794970,2805423,+,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...,1380,533,CAAGGACAAGCTCTTTGACCAGCGGGAGCTCCGCTTTGAGATTGGC...,732
4,ENST00000001146,ENSG00000003137,protein_coding,2,72129238,72148038,-,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...,1539,1252,GCCAGGCGCGGAGCGAGGACAAGGATGGCCGCTTCCATTACCTCCC...,1451


## Random transcript selection

In [74]:
sample_regions = selected_regions.sample(N)
sample_regions.shape

(50000, 12)

In [75]:
seqs = sample_regions[['id', 'chr', 'random_start', 'random_end', 'seq']].copy().reset_index(drop=True)
seqs.head()

,id,chr,random_start,random_end,seq
0,ENST00000594605,19,410,609,AGTCTTGGCCAAAGAGGAGGCGCGCCGTGCACTGGAGACACCCTCG...
1,ENST00000638043,2,2633,2832,ACTGCAGAGTCAGCTACCCATTGGGAGTGATTTTCCTTTTGTTGGC...
2,ENST00000340807,13,574,773,TGACATTTTTTTTCTTGATTCCTTTGTTCATCATGATTGGGTGCTA...
3,ENST00000370508,10,565,764,TCCACGCCTGTGCACACAACCCAACTGGGATTGACCCAACTCCGGA...
4,ENST00000525064,11,164,363,ACTGGAATACCCCACCATTAGTTTACATGCATTATCCAGGGACCGA...


In [76]:
len(seqs.seq.values[0])

200

## Save generated sequences to file

In [78]:
seqs.to_csv(OUTPUT_FILE, index=False)